In [ ]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib import cm
from datetime import datetime
import glob
import os
import json
import pickle
import six
sns.set()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [ ]:
os.chdir("/Users/andyslo/Documents/DATA")
AllCSV  = [i for i in glob.glob('*.{}'.format('csv'))]
AllCSV

In [ ]:
all_dataframes = []                               # list to store each data frame separately
for csv in AllCSV:
    df = pd.read_csv(csv, encoding='latin-1')
    df['country'] = csv[0:2]                      # adding column 'country', each dataset could be identified uniquely
    all_dataframes.append(df)
all_dataframes[0].head()                          # index 0 to 9 for [CA, DE, FR, GB, IN, JP, KR, MX, RU, US] datasets

In [ ]:
for df in all_dataframes:
    # video_id 
    df['video_id'] = df['video_id'].astype('str') 
    
    # trending date
    df['trending_date'] = df['trending_date'].astype('str') 
    date_pieces = (df['trending_date']
                   .str.split('.')
                  )
    df['Year'] = date_pieces.str[0].astype(int)
    df['Day'] = date_pieces.str[1].astype(int)
    df['Month'] = date_pieces.str[2].astype(int)
    updatedyear = []
    for i in range(len(df)) : 
        y = df.loc[i, "Year"]
        newy = y+2000
        updatedyear.append(newy)
    for i in range(len(df)):
        newy = updatedyear[i]
        tr = df.loc[i, "Year"]
        df['Year'].replace(to_replace = tr, value = newy, inplace=True)
    del df['trending_date']
    df['trending_date'] = pd.to_datetime(df[['Year', 'Month', 'Day']], format = "%Y-%m-%d")
    del df['Year']
    del df['Day']
    del df['Month']
    
    #title
    df['title'] = df['title'].astype('str')
    #channel_title
    df['channel_title'] = df['channel_title'].astype('str')
    #category_id
    df['category_id'] = df['category_id'].astype(str) 
    
    #tags
    df['tags'] = df['tags'].astype('str')
    
    # views, likes, dislikes, comment_count are already in correct data types i.e int64
    
    #thumbnail_link
    df['thumbnail_link'] = df['thumbnail_link'].astype('str') 
    
    #description
    df['description'] = df['description'].astype('str')
    
    # Changing comments_disabled, ratings_disabled, video_error_or_removed from bool to categorical
    df['comments_disabled'] = df['comments_disabled'].astype('category') 
    df['ratings_disabled'] = df['ratings_disabled'].astype('category') 
    df['video_error_or_removed'] = df['video_error_or_removed'].astype('category') 
    
    # publish_time 
    df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
for df in all_dataframes:
    df.insert(4, 'publish_date', df['publish_time'].dt.date) # loc, column name, values for column to be inserted
    df['publish_time'] = df['publish_time'].dt.time
# Changing data type for 'publish_date' from object to 'datetime64[ns]'
for df in all_dataframes:
     df['publish_date'] = pd.to_datetime(df['publish_date'], format = "%Y-%m-%d")

In [ ]:
# We can use any index from 0 to 9 inclusive (for each of the 10 dataframes
all_dataframes[1].dtypes

In [ ]:
for df in all_dataframes:
    df.set_index('video_id', inplace=True)

In [ ]:
for df in all_dataframes:
    sns.heatmap(df.isnull(), cbar=False)
    plt.figure()

In [ ]:
combined_df = pd.concat(all_dataframes)

In [ ]:
# Making copy of original dataframe
backup_df = combined_df.reset_index().sort_values('trending_date', ascending=False).set_index('video_id')
# Sorting according to latest trending date while removing duplicates
combined_df = combined_df.reset_index().sort_values('trending_date', ascending=False).drop_duplicates('video_id',keep='first').set_index('video_id')
# Doing the same above operation for each of the individual dataframes in the list we created earlier
for df in all_dataframes:
    df = df.reset_index().sort_values('trending_date', ascending=False).set_index('video_id')
# Printing results
combined_df[['publish_date','publish_time','trending_date', 'country']].head()
# It can be seen that latest publications and trending information is at the top now

In [ ]:
# read file
with open('US_category_id.json', 'r') as f:  # reading one randomly selected json files to make sense of its contents
    data = f.read()
# parse file
obj = json.loads(data)
# printing
obj

In [ ]:
category_id = {}
with open('DE_category_id.json', 'r') as f:
    d = json.load(f)
    for category in d['items']:
        category_id[category['id']] = category['snippet']['title']
combined_df.insert(2, 'category', combined_df['category_id'].map(category_id))
backup_df.insert(2, 'category', backup_df['category_id'].map(category_id))
for df in all_dataframes:
    df.insert(2, 'category', df['category_id'].map(category_id))
# Printing cleaned combined dataframe
combined_df.head(3)

In [ ]:
combined_df['category'].unique()

In [ ]:
# calculating total likes for each category
likesdf = combined_df.groupby('category')['likes'].agg('sum')
# calculating total dislikes for each category
dislikesdf = combined_df.groupby('category')['dislikes'].agg('sum')
# calculating ratios of likes to dislikes
ratiodf = likesdf/dislikesdf 
# most liked category to appear on top
ratiodf = ratiodf.sort_values(ascending=False).reset_index()
# plotting bar chart
ratiodf.columns = ['category','ratio']
plt.subplots(figsize=(10, 15))
sns.barplot(x="ratio", y="category", data=ratiodf,
            label="Likes-Dislikes Ratio", color="r")    

In [ ]:
# Getting names of all countries
countries = []
allcsv = [i for i in glob.glob('*.{}'.format('csv'))]
for csv in allcsv:
    c = csv[0:2]
    countries.append(c)
    
for country in countries:
        tempdf = combined_df[combined_df['country']==country]['category'].value_counts().reset_index()
        ax = sns.barplot(y=tempdf['index'], x=tempdf['category'], data=tempdf, orient='h')
        plt.xlabel("Number of Videos")
        plt.ylabel("Categories")
        plt.title("Catogories of trend videos in " + country)
        plt.figure()
 

In [ ]:
# Calculating days between publish and trending date
temporary = []
for data in all_dataframes:
    temp = data
    temp['timespan'] = (temp['trending_date'] - temp['publish_date']).dt.days
    temporary.append(temp)
# Plotting
to_trending = temporary[0].sample(1000).groupby('video_id').timespan.max() # CA
sns_ax = sns.boxplot(y = to_trending)
_ = sns_ax.set(yscale = "log")
plt.show()
_ = sns.distplot(to_trending.value_counts(),bins='rice',kde=False)


In [ ]:
temp = combined_df
temp = temp.groupby('country')['views','likes','dislikes', 'comment_count'].apply(lambda x: x.astype(int).sum())
temp = temp.sort_values(by='comment_count', ascending=False).head()
temp

In [ ]:
temp = combined_df
temp = temp.groupby('category')['views','likes','dislikes', 'comment_count'].apply(lambda x: x.astype(int).sum())
temp = temp.sort_values(by='comment_count', ascending=False).head()
temp

In [ ]:
# For tags
col = ['views', 'likes', 'dislikes', 'comment_count']
corr = combined_df[col].corr()
corr

In [ ]:
def view_bar(x,y,title):
    plt.figure(figsize = (13,11))
    sns.barplot(x = x, y = y)
    plt.title(title)
    plt.xticks(rotation = 90)
    plt.show()

In [ ]:
x = df.category.value_counts().index
y = df.category.value_counts().values
title = "Categories"
view_bar(x,y,title)

In [ ]:
df.info()

In [ ]:
df[df["category"] == 'Music'][['trending_date', 'video_id']]

In [ ]:
df[df["channel_title"] == 'TheEllenShow'][['channel_title', 'trending_date', 'video_id', 'country']]

In [ ]:
df.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
df[df["channel_title"] == 'TheEllenShow'][['channel_title', 'trending_date', 'video_id', 'country']]

In [ ]:
df.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
df[df["channel_title"] == 'TheEllenShow'].value_counts()

In [ ]:

#CA_tempdf=[CA_tempdf["category"] == 'Music'][['channel_title', 'trending_date']] ---- cutout
CA_tempdf = combined_df[combined_df['country']== 'CA']

CA_tempdf

In [ ]:
combined_df.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
combined_df[combined_df["channel_title"] == 'TheEllenShow'][['channel_title', 'trending_date', 'video_id', 'country']]

In [ ]:
x = df.channel_title.value_counts().head(10).index
y = df.channel_title.value_counts().head(10).values
title = "Top 10 Channels"
view_bar(x,y,title)

In [ ]:
CA_df = pd.read_csv('/Users/andyslo/Documents/DATA/CAvideos.csv')

sort_by_likes = CA_df.sort_values(by ="likes" , ascending = False).drop_duplicates('title', keep = 'first')
x = sort_by_likes['title'].head(10)
y = sort_by_likes['likes'].head(10)
title = "Most liked videos"
view_bar(x,y,title)


In [ ]:
x = CA_df.channel_title.value_counts().head(10).index
y = CA_df.channel_title.value_counts().head(10).values
title = "Top 10 Channels in Canada"
view_bar(x,y,title)

In [ ]:
DE_df = pd.read_csv('/Users/andyslo/Documents/DATA/DEvideos.csv')

sort_by_likes = DE_df.sort_values(by ="likes" , ascending = False).drop_duplicates('title', keep = 'first')
x = sort_by_likes['title'].head(10)
y = sort_by_likes['likes'].head(10)
title = "Most liked videos"
view_bar(x,y,title)

In [ ]:
x = DE_df.channel_title.value_counts().head(10).index
y = DE_df.channel_title.value_counts().head(10).values
title = "Top 10 Channels in Germany"
view_bar(x,y,title)

In [ ]:
CA_df = pd.read_csv('/Users/andyslo/Documents/DATA/CAvideos.csv')
video_stat = CA_df.sort_values('likes', ascending=False)
# removing dublicates
video_stat.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
#printing TOP 50
video_stat.head(50)[['video_id', 'trending_date', 'likes']]

In [ ]:
CA_df = pd.read_csv('/Users/andyslo/Documents/DATA/CAvideos.csv')
video_stat = CA_df.sort_values('dislikes', ascending=False)
# removing dublicates
video_stat.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
#printing TOP 50
video_stat.head(50)[['video_id', 'trending_date', 'dislikes']]

In [ ]:
GB_df = pd.read_csv('/Users/andyslo/Documents/DATA/GBvideos.csv')
video_stat = GB_df.sort_values('likes', ascending=False)
# removing dublicates
video_stat.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
#printing TOP 50
video_stat.head(50)[['video_id', 'trending_date', 'likes']]

In [ ]:
GB_df = pd.read_csv('/Users/andyslo/Documents/DATA/GBvideos.csv')
video_stat = GB_df.sort_values('dislikes', ascending=False)
# removing dublicates
video_stat.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
#printing TOP 50
video_stat.head(50)[['video_id', 'trending_date', 'dislikes']]

In [ ]:
US_df = pd.read_csv('/Users/andyslo/Documents/DATA/USvideos.csv')
video_stat = US_df.sort_values('likes', ascending=False)
# removing dublicates
video_stat.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
#printing TOP 50
video_stat.head(50)[['video_id', 'trending_date', 'likes']]

In [ ]:
US_df = pd.read_csv('/Users/andyslo/Documents/DATA/USvideos.csv')
video_stat = US_df.sort_values('dislikes', ascending=False)
# removing dublicates
video_stat.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
#printing TOP 50
video_stat.head(50)[['video_id', 'trending_date', 'dislikes']]

In [ ]:
x = df.category.value_counts().index
y = df.category.value_counts().valuesdf.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
title = "Categories"
view_bar(x,y,title)

In [ ]:
df.drop_duplicates(subset ="video_id", keep = 'first', inplace = True)
df[df['category'] == 'Music'][['channel_title', 'trending_date', 'video_id', 'country']]


In [ ]:
df['category'].unique()

In [ ]:

combined_df.groupby('category')
print(category_id)